In [1]:
# Install libraries
!pip install -q dash jupyter-dash pandas plotly dash-bootstrap-components
from jupyter_dash import JupyterDash
from dash import dcc, html, Input, Output, dash_table, no_update
import plotly.express as px
import pandas as pd
from google.colab import files
import io
import dash_bootstrap_components as dbc

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 47.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.7/203.7 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 46.8 MB/s eta 0:00:00


In [2]:
# Upload the file
uploaded = files.upload()
file_name = next(iter(uploaded))
df = pd.read_csv(io.BytesIO(uploaded[file_name]))

Saving cleaned_Mumbai Real Estate.csv to cleaned_Mumbai Real Estate.csv


In [29]:
from dash import Dash
app = Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

In [4]:
df['Price'] = df['Price'].astype(str).str.replace(',', '').astype(float)

# Create important
numeric_features = ['Price', 'Price Per Unit', 'Floor No', 'Units Available']
categorical_features = ['Possession Status', 'Developer', 'Approved Authority Name']
binary_features = ['Commercial', 'Meditation Area', 'Multipurpose Courts', 'Recreational Pool']

In [5]:
# App layout
app.layout = dbc.Container([
    html.H1("Real Estate Market Analysis Dashboard", className="text-center my-4"),

    # Filters Row
    dbc.Row([
        dbc.Col([
            html.Label("Price Range (₹):"),
            dcc.RangeSlider(
                id='price-slider',
                min=df['Price'].min(),
                max=df['Price'].max(),
                step=100000,
                marks={i: f"₹{i//100000}L" for i in range(int(df['Price'].min()), int(df['Price'].max())+1, 5000000)},
                value=[df['Price'].min(), df['Price'].max()],
                tooltip={"placement": "bottom", "always_visible": True}
            )
        ], md=12)
    ], className="mb-4"),

    dbc.Row([
        # Left column - filters
        dbc.Col([
            dbc.Card([
                dbc.CardBody([
                    html.H4("Property Filters", className="card-title"),

                    html.Label("Possession Status:", className="mt-3"),
                    dcc.Dropdown(
                        id='possession-dropdown',
                        options=[{'label': status, 'value': status} for status in df['Possession Status'].unique()],
                        multi=True,
                        placeholder="Select Possession Status"
                    ),

                    html.Label("Developer:", className="mt-3"),
                    dcc.Dropdown(
                        id='developer-dropdown',
                        options=[{'label': dev, 'value': dev} for dev in df['Developer'].unique() if pd.notna(dev)],
                        multi=True,
                        placeholder="Select Developer"
                    ),

                    html.Label("Floor Range:", className="mt-3"),
                    dcc.RangeSlider(
                        id='floor-slider',
                        min=0,
                        max=100,
                        step=1,
                        marks={i: str(i) for i in range(0, 101, 10)},
                        value=[0, 100]
                    ),

                    html.Label("Amenities:", className="mt-3"),
                    dcc.Checklist(
                        id='amenities-checklist',
                        options=[
                            {'label': ' Commercial', 'value': 'Commercial'},
                            {'label': ' Meditation Area', 'value': 'Meditation Area'},
                            {'label': ' Multipurpose Courts', 'value': 'Multipurpose Courts'},
                            {'label': ' Recreational Pool', 'value': 'Recreational Pool'}
                        ],
                        value=[],
                        inline=False
                    ),

                    dbc.Button("Reset Filters", id="reset-btn", color="secondary", className="mt-3")
                ])
            ])
        ], md=3),

        # Right column - visualizations
        dbc.Col([
            dbc.Tabs([
                dbc.Tab([
                    dcc.Graph(id='price-distribution')
                ], label="Price Analysis"),

                dbc.Tab([
                    dcc.Graph(id='possession-plot')
                ], label="Possession Status"),

                dbc.Tab([
                    dcc.Graph(id='developer-plot')
                ], label="Developer Analysis"),

                dbc.Tab([
                    dash_table.DataTable(
                        id='data-table',
                        columns=[{"name": i, "id": i} for i in ['Possession Status', 'Developer', 'Floor No', 'Price', 'Price Per Unit']],
                        page_size=10,
                        style_table={'overflowX': 'auto'},
                        style_cell={
                            'minWidth': '100px', 'width': '100px', 'maxWidth': '100px',
                            'whiteSpace': 'normal'
                        },
                        filter_action="native",
                        sort_action="native",
                        style_header={
                            'backgroundColor': '#2c3e50',
                            'color': 'white',
                            'fontWeight': 'bold'
                        }
                    )
                ], label="Property Listings")
            ])
        ], md=9)
    ])
], fluid=True)

In [6]:
# Callbacks
@app.callback(
    [Output('price-distribution', 'figure'),
     Output('possession-plot', 'figure'),
     Output('developer-plot', 'figure'),
     Output('data-table', 'data')],
    [Input('price-slider', 'value'),
     Input('possession-dropdown', 'value'),
     Input('developer-dropdown', 'value'),
     Input('floor-slider', 'value'),
     Input('amenities-checklist', 'value'),
     Input('reset-btn', 'n_clicks')],
    prevent_initial_call=True
)
def update_output(price_range, possession_status, developers, floor_range, amenities, reset_clicks):
    # Filter data
    filtered_df = df[
        (df['Price'].between(price_range[0], price_range[1])) &
        (df['Floor No'].between(floor_range[0], floor_range[1]))
    ]

    if possession_status:
        filtered_df = filtered_df[filtered_df['Possession Status'].isin(possession_status)]
    if developers:
        filtered_df = filtered_df[filtered_df['Developer'].isin(developers)]

    for amenity in amenities:
        filtered_df = filtered_df[filtered_df[amenity] == 1]

    # Create figures
    price_fig = px.histogram(
        filtered_df,
        x='Price',
        nbins=30,
        title='Price Distribution',
        labels={'Price': 'Price (₹)'},
        color_discrete_sequence=['#3498db']
    )

    possession_fig = px.box(
        filtered_df,
        x='Possession Status',
        y='Price',
        title='Price by Possession Status',
        labels={'Price': 'Price (₹)'}
    )

    developer_fig = px.box(
        filtered_df,
        x='Developer',
        y='Price',
        title='Price by Developer',
        labels={'Price': 'Price (₹)'}
    )
    developer_fig.update_layout(xaxis={'categoryorder':'total descending'})

    # Prepare table data
    table_data = filtered_df[['Possession Status', 'Developer', 'Floor No', 'Price', 'Price Per Unit']].to_dict('records')

    return price_fig, possession_fig, developer_fig, table_data

In [30]:
# Reset button callback
@app.callback(
    [Output('price-slider', 'value'),
     Output('possession-dropdown', 'value'),
     Output('developer-dropdown', 'value'),
     Output('floor-slider', 'value'),
     Output('amenities-checklist', 'value')],
    Input('reset-btn', 'n_clicks')
)
def reset_filters(n_clicks):
    if n_clicks:
        return ([df['Price'].min(), df['Price'].max()],
                None,
                None,
                [0, 100],
                [])
    return no_update

# Run the app
if __name__ == '__main__':
    app.run(mode='inline', port=8050, debug=False)

<IPython.core.display.Javascript object>